# Emotion detection on video file

https://www.youtube.com/watch?v=44elcDAJWyo

### Imports

In [36]:
import os
import cv2
from PIL import Image
import matplotlib.pyplot as plt

import torch
from ultralytics import YOLO

from emotions_utils import inf_transforms

In [2]:
VIDEO_PATH = "emotions_video.mp4"
FRAME_SIZE = 640

Face detection

source: https://github.com/akanametov/yolov8-face

In [ ]:
detection_model = YOLO("yolov8n-face.pt")
detection_model = detection_model.to("cuda")

In [29]:
def detect_faces(frame, verbose=False):
    return detection_model.predict(frame, conf=0.5, verbose=verbose)

Create camera

In [79]:
cam = cv2.VideoCapture(VIDEO_PATH)

if not cam.isOpened():
    raise Exception("Can not read input")

Main algorithm

In [80]:
while True:
    success, frame = cam.read()
    if not success:
        raise Exception("Error during reading the frames")
    
    frame = cv2.resize(frame, (FRAME_SIZE, FRAME_SIZE))
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    faces = detect_faces(frame, verbose=False)[0]
    for bbox in faces.boxes:
        x1, y1, x2, y2 = bbox.xyxy[0]
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        h, w = y2 - y1, x2 - x1
        region_of_interest = image_rgb[y1:y1+h, x1:x1+w]
        Image.fromarray(region_of_interest)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0))

    cv2.imshow("Camera video", frame)
    if cv2.waitKey(1) == ord("q"):
        break

cam.release()
cv2.destroyAllWindows()